# Hybrid Model for Movie Recommendation
## Oleh : Febi Andika Dani Fajar Suryawan


## Studi Kasus
Diberikan data yang berisi 10 judul film beserta genre, director dan actors dari masing masing film beserta data rating dari film berdasarkan penilaian dari 24 orang. Akan dibuat suatu rekomendasi film dimana jika seseorang telah selesai menonton suatu film, maka akan direkomendasikan daftar film yang memiliki kemiripan yang tinggi dengan film yang telah ditonton dengan mempertimbangkan rating penilaian film yang telah diberikan.
<br>
<br>
Pada model Hybrid digunakan gabungan dua buah model yaitu Collaboration Filtering dan Content Based model.

## Preparasi Data

In [1]:
# Memanggil library yang dibutuhkan
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
#from recommendation_data import dataset
from math import sqrt

In [8]:
# Memanggil dataset yang diperlukan
df=pd.read_csv('film.csv',delimiter=';')
df

,Title,Genre,Director,Actors
0,Ada Apa dengan Cinta 2,"Romance, Drama",Riri Riza,"Nicholas Saputra, Dian Sastrowardoyo, Adinia W..."
1,Gundala,"Action, Drama",Joko Anwar,"Tara Basro, Abimana Aryasatya, Lukman Sardi"
2,Dilan 1991,"Romance, Drama",Pidi Baiq,"Iqbaal Ramadhan, Vanesha Prescilla"
3,Bumi Manusia,"Drama, History",Hanung Bramantyo,"Iqbaal Ramadhan, Mawar Eva"
4,Dua Garis Biru,Drama,Ginatri Noer,"Adhisty Zara, Angga Yunanda"
5,Avengers: End Game,"Action, Fiction",Joe Russo,"Robert Downey, Chris Evans, Chris Hemworth"
6,The Lion King,"Drama, Adventure",Jon Favreau,"Donald Glover, Beyonce"
7,Aladdin,"Romance, Fantasy",Guy Ritchie,"Naomi Scott, Mena Massoud"
8,Spiderman: Far From Home,"Fantasy, Fiction",Jon Watts,"Tom Holland, Zendaya"
9,Captain Marvel,Fiction,Anna Boden,"Brie Larson, Samuel Jackson"


In [9]:
# Mengambil kolom Title, Genre, Director, Actors dari dataset
df=df[['Title','Genre','Director','Actors']]
df.head()

,Title,Genre,Director,Actors
0,Ada Apa dengan Cinta 2,"Romance, Drama",Riri Riza,"Nicholas Saputra, Dian Sastrowardoyo, Adinia W..."
1,Gundala,"Action, Drama",Joko Anwar,"Tara Basro, Abimana Aryasatya, Lukman Sardi"
2,Dilan 1991,"Romance, Drama",Pidi Baiq,"Iqbaal Ramadhan, Vanesha Prescilla"
3,Bumi Manusia,"Drama, History",Hanung Bramantyo,"Iqbaal Ramadhan, Mawar Eva"
4,Dua Garis Biru,Drama,Ginatri Noer,"Adhisty Zara, Angga Yunanda"


In [10]:
#Membuang koma diantara nama lengkap aktor dan mengambil 2 nama awal
df['Actors']=df['Actors'].map(lambda x: x.split(',')[:2])

#Membuang koma diantara Genre
df['Genre']=df['Genre'].map(lambda x: x.lower().split(','))
df['Director']=df['Director'].map(lambda x: x.split(' '))

for index, row in df.iterrows():
    row['Actors'] = [x.lower().replace(' ','') for x in row['Actors']]
    row['Director'] = ''.join(row['Director']).lower()

In [11]:
# Membuat kolom Title menjadi index
df.set_index('Title', inplace = True)
df.head()

,Genre,Director,Actors
Title,,,
Ada Apa dengan Cinta 2,"[romance, drama]",riririza,"[nicholassaputra, diansastrowardoyo]"
Gundala,"[action, drama]",jokoanwar,"[tarabasro, abimanaaryasatya]"
Dilan 1991,"[romance, drama]",pidibaiq,"[iqbaalramadhan, vaneshaprescilla]"
Bumi Manusia,"[drama, history]",hanungbramantyo,"[iqbaalramadhan, mawareva]"
Dua Garis Biru,[drama],ginatrinoer,"[adhistyzara, anggayunanda]"


In [12]:
# Menggabungkan kolom Genre, Director, Actors menjadi satu 
df['words'] = ''
columns = df.columns
for index, row in df.iterrows():
    words = ''
    for col in columns:
        if col != 'Director':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '
    row['words'] = words
    
df.drop(columns = [col for col in df.columns if col!= 'words'], inplace = True)
df.head()

,words
Title,
Ada Apa dengan Cinta 2,romance drama riririza nicholassaputra diansa...
Gundala,action drama jokoanwar tarabasro abimanaaryas...
Dilan 1991,romance drama pidibaiq iqbaalramadhan vanesha...
Bumi Manusia,drama history hanungbramantyo iqbaalramadhan ...
Dua Garis Biru,drama ginatrinoer adhistyzara anggayunanda


In [44]:
# Memanggil dataset
data=pd.read_csv('Film_Rating.csv',delimiter=';')
data=data.set_index('Nama Anda')
data.head()

,Ada Apa dengan Cinta 2,Gundala,Dilan 1991,Bumi Manusia,Dua Garis Biru,Avengers: End Game,The Lion King,Aladdin,Spiderman: Far From Home,Captain Marvel
Nama Anda,,,,,,,,,,
Hania,3,5,4,4,4,0,0,0,0,0
Topik Zulkarnain,0,0,0,0,0,5,5,0,4,2
AhokTemanFirli,0,0,0,0,0,3,0,0,0,4
franadek,4,4,4,5,3,5,4,5,4,4
OM INDRA,3,0,2,0,5,5,0,1,5,5


In [33]:
# Membuat data rating menjadi dictionary
dataset=data.to_dict(orient='index')
for name in dataset:
    dicto={}
    for movie in dataset[name]:
        if dataset[name][movie]!=0:
            dicto[movie]=dataset[name][movie]
    dataset[name]=dicto
dataset

{'ANI': {'Ada Apa dengan Cinta 2': 4,
  'Aladdin': 4,
  'Bumi Manusia ': 5,
  'Captain Marvel': 4,
  'Dilan 1991': 4,
  'Spiderman: Far From Home': 3},
 'AhokTemanFirli': {'Avengers: End Game': 3, 'Captain Marvel': 4},
 'Damar Teman Firli': {'Ada Apa dengan Cinta 2': 5,
  'Avengers: End Game': 5,
  'Spiderman: Far From Home': 5},
 'Dpv': {'Ada Apa dengan Cinta 2': 5,
  'Avengers: End Game': 5,
  'Captain Marvel': 5,
  'Dilan 1991': 4,
  'Gundala': 4,
  'Spiderman: Far From Home': 5},
 'Febi ganteng gak ada obat': {'Ada Apa dengan Cinta 2': 4,
  'Aladdin': 5,
  'Avengers: End Game': 5,
  'Captain Marvel': 4,
  'Dilan 1991': 4,
  'Gundala': 3,
  'Spiderman: Far From Home': 5},
 'Genjeh': {'Ada Apa dengan Cinta 2': 5,
  'Aladdin': 5,
  'Avengers: End Game': 5,
  'Captain Marvel': 4,
  'Dilan 1991': 3,
  'Gundala': 4,
  'Spiderman: Far From Home': 4},
 'Hania': {'Ada Apa dengan Cinta 2': 3,
  'Bumi Manusia ': 4,
  'Dilan 1991': 4,
  'Dua Garis Biru': 4,
  'Gundala': 5},
 'Indra 1991 SM': {

## Content Based
Dalam membuat model rekomendasi film berdasarkan konten, akan digunakan cosine similarity matrix untuk menghitung kemiripan diantara tiap film. Untuk menghitung cosine similarity matrix digunakan fungsi CountVectorizer untuk membangkitkan count matrix yang berisi jumlah kata dalam kolom words setiap baris berdasarkan daftar kata yang unik dari kumpulan kata di kolom words.

In [26]:
# Membangkitkan count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['words'])

In [27]:
# Membangun cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.       , 0.2      , 0.4      , 0.2      , 0.2236068, 0.       ,
        0.2      , 0.2      , 0.       , 0.       ],
       [0.2      , 1.       , 0.2      , 0.2      , 0.2236068, 0.2      ,
        0.2      , 0.       , 0.       , 0.       ],
       [0.4      , 0.2      , 1.       , 0.4      , 0.2236068, 0.       ,
        0.2      , 0.2      , 0.       , 0.       ],
       [0.2      , 0.2      , 0.4      , 1.       , 0.2236068, 0.       ,
        0.2      , 0.       , 0.       , 0.       ],
       [0.2236068, 0.2236068, 0.2236068, 0.2236068, 1.       , 0.       ,
        0.2236068, 0.       , 0.       , 0.       ],
       [0.       , 0.2      , 0.       , 0.       , 0.       , 1.       ,
        0.       , 0.       , 0.2      , 0.2236068],
       [0.2      , 0.2      , 0.2      , 0.2      , 0.2236068, 0.       ,
        1.       , 0.       , 0.       , 0.       ],
       [0.2      , 0.       , 0.2      , 0.       , 0.       , 0.       ,
        0.       , 1.       , 0.2    

Selanjutnya, dibuat fungsi untuk membuat daftar film berdasarkan kemiripannya dengan film lain. Misalkan terdapat film A, maka akan direkomendasikan daftar 3 film yang memiliki kemiripan paling tinggi dengan film A berdasarkan cosine similarity matrix yang telah dibuat.

In [28]:
# fungsi untuk mengambil judul film sebagai input dan mengeluarkan 3 rekomendasi film 
indices = pd.Series(df.index)
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    # Mengambil indeks film yang sesuai dengan judul film
    idx = indices[indices == title].index[0]

    # Membuat series yang berisi similarity scores
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # Mengambil indeks dari 3 film yang memiliki kemiripan paling tinggi
    top_10_indexes = list(score_series.iloc[1:4].index)
    
    # Membuat list berisi judul film yang direkomendasikan
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

## Collaborating Filtering

### Similarity Score
Pada penghitungan similarity score akan digunakan jarak euclidean dari rating yang diberikan dari dua orang yang memberikan rating terhadap suatu film. Nilai similarity score sendiri adalah 1/(1+euclidean).

In [15]:
# Membuat fungsi untuk menghitung nilai euclidean distance dari orang pertama dan kedua
def similarity_score(person1,person2):
    
    # Mengambil item yang telah diberi rating oleh orang pertama dan orang kedua
    both_viewed = {}

    for item in dataset[person1]:
        if item in dataset[person2]:
            both_viewed[item] = 1
            
        if len(both_viewed) == 0:
            return 0

        # Menghitung Euclidean distance
        sum_of_eclidean_distance = []

        for item in dataset[person1]:
            if item in dataset[person2]:
                sum_of_eclidean_distance.append(pow(dataset[person1][item] - dataset[person2][item], 2))
        sum_of_eclidean_distance = sum(sum_of_eclidean_distance)
        
        return 1/(1+sqrt(sum_of_eclidean_distance))

### Person Correlation
Pada penghitungan person correlation akan digunakan korelasi diantara dua orang yang memberikan rating terhadap suatu film. Semakin besar nilai person correlation berarti kedua orang memiliki preferensi/ kesukaan yang sama terhadap suatu film, begitupun sebaliknya.

In [16]:
# Membuat fungsi untuk menghitung nilai person correlation dari orang pertama dan kedua
def person_correlation(person1, person2):

   # Mengambil item yang keduanya telah diberi rating
    both_rated = {}
    for item in dataset[person1]:
        if item in dataset[person2]:
            both_rated[item] = 1

    number_of_ratings = len(both_rated)

    if number_of_ratings == 0:
        return 0

    # Menjumlahkan semua preferensi dari setiap user
    person1_preferences_sum = sum([dataset[person1][item] for item in both_rated])
    person2_preferences_sum = sum([dataset[person2][item] for item in both_rated])

    # Jumlah kuadrat dari preferensi dari setiap user
    person1_square_preferences_sum = sum([pow(dataset[person1][item],2) for item in both_rated])
    person2_square_preferences_sum = sum([pow(dataset[person2][item],2) for item in both_rated])

    # Menjumlahkan nilai produk dari preferensi keduanya untuk setiap item
    product_sum_of_both_users = sum([dataset[person1][item] * dataset[person2][item] for item in both_rated])

    # Menghitung pearson score
    numerator_value = product_sum_of_both_users - (person1_preferences_sum*person2_preferences_sum/number_of_ratings)
    denominator_value = sqrt((person1_square_preferences_sum - pow(person1_preferences_sum,2)/number_of_ratings) * (person2_square_preferences_sum -pow(person2_preferences_sum,2)/number_of_ratings))

    if denominator_value == 0:
        return 0
    else:
        r = numerator_value / denominator_value
        return r

Pada model rekomendasi film berikut digunakan person correlation untuk memberikan score/nilai rekomendasi terhadap suatu film.

In [25]:
# Fungsi untuk menentukan banyaknya users (orang yang mirip) dengan orang tersebut
def most_similar_users(person, number_of_users):

    scores = [(person_correlation(person, other_person), other_person) for other_person in dataset if other_person != person]
    scores.sort()
    scores.reverse()
    return scores[0:number_of_users]

In [24]:
# Membuat fungsi yang memberikan rekomendasi film kepada seseorang
def user_recommendations(person):

    totals = {}
    simSums = {}
    rankings_list =[]
    for other in dataset:
        if other == person:
            continue
        sim = person_correlation(person,other)
        if sim <=0: 
            continue
        for item in dataset[other]:

            # Hanya menilai film yang belum ditonton
            if item not in dataset[person] or dataset[person][item] == 0:
                totals.setdefault(item,0)
                totals[item] += dataset[other][item]* sim
                simSums.setdefault(item,0)
                simSums[item]+= sim

    rankings = [(total/simSums[item],item) for item,total in totals.items()]
    rankings.sort()
    rankings.reverse()
    recommendations_list = [recommend_item for score,recommend_item in rankings]
    return recommendations_list,rankings

## Hybrid Model
Sebagai contoh, disini ANI baru saja selesai menonton film Captain Marvel. Andaikan ANI memiliki keinginan untuk menonton film lain yang belum pernah ia tonton yang memiliki kemiripan dengan film Captain Marvel yang baru ia tonton, maka diperoleh daftar rekomendasi film yang memiliki kemiripan dengan film Captain Marvel yang sudah ANI tonton dan memiliki Recommendation Score dari yang terbesar berdasarkan penilaian rating dari orang lain adalah sebagai berikut.

In [43]:
# Memasukkan nama user dan judul film yang pernah ditonton oleh user
name = input("Masukkan nama: ")
title= input("Masukkan judul film yang sudah pernah ditonton: ")

# Memanggil fungsi untuk mengeluarkan rekomendasi film
film_recommendation=recommendations(title)
user_recommendation=user_recommendations(name)

# Membuat dataframe hasil rekomendasi film
out=pd.DataFrame(user_recommendation[1],columns=['Recommendation Score','Film Title'])
out=out[['Film Title','Recommendation Score']]
for i in film_recommendation:
    if i in np.array(out['Film Title']):
        recommended_movies=out[out['Film Title']==i]

# Mengeluarkan rekomendasi film
print('Film yang direkomendasikan untuk {} adalah'.format(name))
print(recommended_movies)

Masukkan nama: ANI
Masukkan judul film yang sudah pernah ditonton: Captain Marvel
Film yang direkomendasikan untuk ANI adalah
           Film Title  Recommendation Score
0  Avengers: End Game                   5.0


**Catatan :** Pada hybrid model, alangkah baiknya digunakan data film yang relatif lebih banyak. Hal ini dapat membuat proses melihat kemiripan suatu film jauh lebih akurat dan baik (sudah dibuktikan). Pada model ini hanya digunakan data yang telah dibuat dalam proses pembelajaran kelas saja.